# TNT Loop-Bridge: Startup Shear

## Objectives

- Fit TNT Loop-Bridge model to startup shear data
- Understand bridge fraction evolution during transient flow
- Analyze force-dependent stress overshoot
- Quantify bridge stretching vs detachment dynamics
- Perform Bayesian inference for parameter uncertainty

## Setup

In [ ]:
import os
import sys
import time

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax

import numpy as np
import matplotlib.pyplot as plt
import arviz as az

from rheojax.core.jax_config import safe_import_jax
jax, jnp = safe_import_jax()
from rheojax.core.jax_config import verify_float64
verify_float64()

from rheojax.models.tnt import TNTLoopBridge

sys.path.insert(0, os.path.join("..", "utils"))
from tnt_tutorial_utils import (
    load_ml_ikh_flow_curve,
    load_pnas_startup,
    load_laponite_relaxation,
    load_ml_ikh_creep,
    load_epstein_saos,
    load_pnas_laos,
    compute_fit_quality,
    print_convergence_summary,
    print_parameter_comparison,
    save_tnt_results,
    get_tnt_loop_bridge_param_names,
    plot_loop_bridge_fraction,
    plot_bell_nu_sweep,
    compute_maxwell_moduli,
    compute_bell_effective_lifetime,
    print_nu_interpretation,
)

param_names = get_tnt_loop_bridge_param_names()

## Theory: Startup Shear Dynamics

### Physical Picture

During startup from rest:
1. **Initial state**: Bridges at equilibrium (f_B = f_B_eq)
2. **Early times**: Bridges stretch elastically → stress increases
3. **Intermediate**: Force on bridges increases → Bell detachment accelerates
4. **Overshoot**: Peak stress when stretching rate = detachment rate
5. **Late times**: Bridge fraction decreases to steady state → stress plateau

### Governing Equations

**Bridge Fraction Evolution:**
```
df_B/dt = (1 - f_B)/tau_a - f_B * exp(nu * gamma_dot * tau_b) / tau_b
```

**Stress Evolution (Maxwell backbone):**
```
sigma(t) = f_B(t) * G * int_0^t gamma_dot * exp(-(t-s)/tau_b) ds + eta_s * gamma_dot
```

For constant gamma_dot:
```
sigma(t) = f_B(t) * G * gamma_dot * tau_b * [1 - exp(-t/tau_b)] + eta_s * gamma_dot
```

### Stress Overshoot Mechanism

The overshoot arises from competition between:
- **Stretching**: Builds stress via Maxwell element
- **Detachment**: Reduces stress via f_B decrease

Peak occurs when df_B/dt is most negative (fastest detachment).

### Force-Dependent Features

- Higher nu → stronger overshoot (more force sensitivity)
- Higher gamma_dot → earlier overshoot (faster force buildup)
- Peak stress scales with G * f_B_eq
- Overshoot time ~ tau_b / (1 + nu * gamma_dot * tau_b)

## Load Startup Data

In [ ]:
time_data, stress = load_pnas_startup(gamma_dot=1.0)

print(f"Data points: {len(time_data)}")
print(f"Time range: {time_data.min():.2e} - {time_data.max():.2e} s")
print(f"Stress range: {stress.min():.2f} - {stress.max():.2f} Pa")
print(f"Shear rate: 1.0 1/s")

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(time_data, stress, 'o', label='Data', markersize=6)
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Stress (Pa)', fontsize=12)
ax.set_title('Startup Shear Data (γ̇ = 1.0 1/s)', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
display(fig)
plt.close(fig)

## NLSQ Fitting

In [ ]:
model = TNTLoopBridge()

print("Starting NLSQ fit...")
t_start = time.time()

nlsq_result = model.fit(time_data, stress, test_mode='startup', gamma_dot=1.0)

t_nlsq = time.time() - t_start
print(f"\nNLSQ fit completed in {t_nlsq:.2f} seconds")
print(f"\nFitted parameters:")
for name in param_names:
    value = getattr(model, name)
    print(f"  {name}: {value:.4e}")

metrics = compute_fit_quality(time_data, stress, model, 'startup', gamma_dot=1.0)
print(f"\nFit quality:")
print(f"  R²: {metrics['r_squared']:.6f}")
print(f"  RMSE: {metrics['rmse']:.4e}")
print(f"  Max relative error: {metrics['max_rel_error']:.2f}%")

## NLSQ Fit Visualization

In [ ]:
time_pred = jnp.linspace(time_data.min(), time_data.max(), 200)
stress_pred = model.predict(time_pred, test_mode='startup', gamma_dot=1.0)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Startup curve
ax1.plot(time_data, stress, 'o', label='Data', markersize=6, alpha=0.7)
ax1.plot(time_pred, stress_pred, '-', label='NLSQ Fit', linewidth=2)
ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel('Stress (Pa)', fontsize=12)
ax1.set_title(f'Startup Fit (R² = {metrics["r_squared"]:.4f})', fontsize=14)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Residuals
stress_fit = model.predict(time_data, test_mode='startup', gamma_dot=1.0)
residuals = (stress - stress_fit) / stress * 100
ax2.plot(time_data, residuals, 'o', markersize=6)
ax2.axhline(0, color='k', linestyle='--', alpha=0.3)
ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel('Relative Error (%)', fontsize=12)
ax2.set_title('Fit Residuals', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

## Physical Analysis: Bridge Fraction Evolution

In [ ]:
# Analytical estimate of bridge fraction during startup
gamma_dot = 1.0
k_detach = jnp.exp(model.nu * gamma_dot * model.tau_b) / model.tau_b
k_attach = 1.0 / model.tau_a
f_B_ss = k_attach / (k_attach + k_detach)
lambda_eff = k_attach + k_detach
f_B_t = f_B_ss + (model.f_B_eq - f_B_ss) * jnp.exp(-lambda_eff * time_pred)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Bridge fraction evolution
ax1.plot(time_pred, f_B_t, '-', linewidth=2)
ax1.axhline(model.f_B_eq, color='r', linestyle='--', alpha=0.5, label=f'f_B_eq = {model.f_B_eq:.4f}')
ax1.axhline(f_B_ss, color='g', linestyle='--', alpha=0.5, label=f'f_B_ss = {f_B_ss:.4f}')
ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel('Bridge Fraction f_B', fontsize=12)
ax1.set_title('Bridge Fraction Evolution During Startup', fontsize=14)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_ylim([0, 1])

# Effective modulus evolution
G_eff_t = f_B_t * model.G
ax2.plot(time_pred, G_eff_t, '-', linewidth=2)
ax2.axhline(model.f_B_eq * model.G, color='r', linestyle='--', alpha=0.5, label='G_eff(0)')
ax2.axhline(f_B_ss * model.G, color='g', linestyle='--', alpha=0.5, label='G_eff(∞)')
ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel('Effective Modulus (Pa)', fontsize=12)
ax2.set_title('Effective Modulus Evolution', fontsize=14)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

print(f"\nBridge fraction dynamics:")
print(f"  Initial f_B: {model.f_B_eq:.4f}")
print(f"  Steady-state f_B: {f_B_ss:.4f}")
print(f"  Reduction: {(1 - f_B_ss/model.f_B_eq)*100:.2f}%")
print(f"  Characteristic decay time: {1.0/lambda_eff:.4e} s")

## Physical Analysis: Stress Overshoot

In [ ]:
# Locate stress overshoot
peak_idx = jnp.argmax(stress_pred)
peak_time = time_pred[peak_idx]
peak_stress = stress_pred[peak_idx]
steady_stress = stress_pred[-1]
overshoot_ratio = peak_stress / steady_stress

fig, ax = plt.subplots(figsize=(10, 7))
ax.plot(time_pred, stress_pred, '-', linewidth=2, label='Model Prediction')
ax.plot(peak_time, peak_stress, 'ro', markersize=10, label=f'Peak: {peak_stress:.2f} Pa at {peak_time:.3f} s')
ax.axhline(steady_stress, color='g', linestyle='--', alpha=0.5, label=f'Steady State: {steady_stress:.2f} Pa')
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Stress (Pa)', fontsize=12)
ax.set_title(f'Stress Overshoot Analysis (Ratio = {overshoot_ratio:.3f})', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
display(fig)
plt.close(fig)

print(f"\nStress overshoot analysis:")
print(f"  Peak stress: {peak_stress:.4e} Pa")
print(f"  Peak time: {peak_time:.4e} s")
print(f"  Steady-state stress: {steady_stress:.4e} Pa")
print(f"  Overshoot ratio: {overshoot_ratio:.4f}")
print(f"  Peak time / tau_b: {peak_time / model.tau_b:.4f}")

## Physical Analysis: Force-Dependent Overshoot

In [ ]:
# Sweep shear rate to show overshoot variation
gamma_dot_sweep = jnp.array([0.1, 1.0, 10.0, 100.0])
time_sweep = jnp.linspace(0, 10.0 * model.tau_b, 200)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

for gd in gamma_dot_sweep:
    stress_sweep = model.predict(time_sweep, test_mode='startup', gamma_dot=gd)
    ax1.plot(time_sweep, stress_sweep, '-', linewidth=2, label=f'γ̇ = {gd:.1f} 1/s')

ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel('Stress (Pa)', fontsize=12)
ax1.set_title('Startup Curves at Different Shear Rates', fontsize=14)
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Extract peak stress and time for each shear rate
peak_stresses = []
peak_times = []
for gd in gamma_dot_sweep:
    stress_sweep = model.predict(time_sweep, test_mode='startup', gamma_dot=gd)
    peak_idx = jnp.argmax(stress_sweep)
    peak_stresses.append(stress_sweep[peak_idx])
    peak_times.append(time_sweep[peak_idx])

ax2.loglog(gamma_dot_sweep, peak_stresses, 'o-', linewidth=2, markersize=8, label='Peak Stress')
ax2.set_xlabel('Shear Rate (1/s)', fontsize=12)
ax2.set_ylabel('Peak Stress (Pa)', fontsize=12)
ax2.set_title('Peak Stress vs Shear Rate', fontsize=14)
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

print(f"\nShear rate dependence:")
for i, gd in enumerate(gamma_dot_sweep):
    print(f"  γ̇ = {gd:.1f} 1/s: peak = {peak_stresses[i]:.4e} Pa at t = {peak_times[i]:.4e} s")

## Bayesian Inference

In [ ]:
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

print(f"Starting Bayesian inference with NUTS...")
print(f"  Warmup: {NUM_WARMUP}, Samples: {NUM_SAMPLES}, Chains: {NUM_CHAINS}")

t_start = time.time()
bayes_result = model.fit_bayesian(
    time_data, stress,
    test_mode='startup',
    gamma_dot=1.0,
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    seed=42
)
t_bayes = time.time() - t_start

print(f"\nBayesian inference completed in {t_bayes:.2f} seconds")
print(f"Speedup vs NLSQ: {t_bayes/t_nlsq:.1f}x slower (includes MCMC overhead)")

## Convergence Diagnostics

In [ ]:
print_convergence_summary(bayes_result, param_names)

## Parameter Comparison: NLSQ vs Bayesian

In [ ]:
print_parameter_comparison(model, bayes_result.posterior_samples, param_names)

## ArviZ: Trace Plot

In [ ]:
idata = az.from_dict(posterior=bayes_result.posterior_samples)

fig = az.plot_trace(idata, var_names=param_names, compact=False, backend_kwargs={'figsize': (12, 10)})
plt.tight_layout()
display(fig)
plt.close()

## ArviZ: Posterior Distributions

In [ ]:
fig = az.plot_posterior(idata, var_names=param_names, hdi_prob=0.95, backend_kwargs={'figsize': (12, 8)})
plt.tight_layout()
display(fig)
plt.close()

## ArviZ: Pair Plot

In [ ]:
fig = az.plot_pair(
    idata,
    var_names=param_names,
    kind='kde',
    marginals=True,
    backend_kwargs={'figsize': (14, 14)}
)
plt.tight_layout()
display(fig)
plt.close()

## Posterior Predictive

In [ ]:
posterior = bayes_result.posterior_samples
n_draws = 200
indices = np.random.choice(NUM_SAMPLES, size=n_draws, replace=False)

predictions = []
for i in indices:
    params_i = jnp.array([posterior[name][i] for name in param_names])
    pred = model.model_function(jnp.array(time_pred), params_i, test_mode='startup', gamma_dot=1.0)
    predictions.append(np.array(pred))

predictions = np.array(predictions)
pred_mean = predictions.mean(axis=0)
pred_lower = np.percentile(predictions, 2.5, axis=0)
pred_upper = np.percentile(predictions, 97.5, axis=0)

fig, ax = plt.subplots(figsize=(10, 7))
ax.plot(time_data, stress, 'o', label='Data', markersize=6, alpha=0.7, zorder=3)
ax.plot(time_pred, pred_mean, '-', label='Posterior Mean', linewidth=2, zorder=2)
ax.fill_between(time_pred, pred_lower, pred_upper, alpha=0.3, label='95% Credible Interval', zorder=1)
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Stress (Pa)', fontsize=12)
ax.set_title('Posterior Predictive Distribution', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
display(fig)
plt.close(fig)

## Physical Interpretation

In [ ]:
print("\n=== Physical Interpretation ===")
print(f"\n1. Material Properties:")
print(f"   - Plateau modulus G: {model.G:.4e} Pa")
print(f"   - Equilibrium bridge fraction: {model.f_B_eq:.4f}")
print(f"   - Initial effective modulus: {model.G * model.f_B_eq:.4e} Pa")

print(f"\n2. Startup Dynamics:")
print(f"   - Bridge detachment time tau_b: {model.tau_b:.4e} s")
print(f"   - Loop attachment time tau_a: {model.tau_a:.4e} s")
print(f"   - Effective decay time: {1.0/lambda_eff:.4e} s")
print(f"   - Bridge fraction reduction: {(1 - f_B_ss/model.f_B_eq)*100:.2f}%")

print(f"\n3. Stress Overshoot:")
print(f"   - Peak stress: {peak_stress:.4e} Pa")
print(f"   - Peak time: {peak_time:.4e} s ({peak_time/model.tau_b:.2f} * tau_b)")
print(f"   - Steady-state stress: {steady_stress:.4e} Pa")
print(f"   - Overshoot ratio: {overshoot_ratio:.4f}")

print(f"\n4. Bell Detachment:")
print(f"   - Nu parameter: {model.nu:.4f}")
print(f"   - Force factor at γ̇ = 1.0 1/s: {jnp.exp(model.nu * 1.0 * model.tau_b):.4f}")
print(f"   - Effective detachment rate: {k_detach:.4e} 1/s")
print(f"   - Rate enhancement: {k_detach * model.tau_b:.4f}x")

print(f"\n5. Timescale Separation:")
print(f"   - tau_a/tau_b ratio: {model.tau_a/model.tau_b:.4f}")
if model.tau_a > model.tau_b:
    print(f"   - Slow attachment dominates re-equilibration")
else:
    print(f"   - Fast attachment allows rapid re-equilibration")

## Save Results

In [ ]:
save_tnt_results(model, bayes_result, "loop_bridge", "startup", param_names)
print("Results saved to reference_outputs/tnt/loop_bridge_startup_results.npz")

## Key Takeaways

1. **Transient Kinetics**: Startup captures bridge fraction evolution from equilibrium to steady state

2. **Stress Overshoot**: Peak arises from competition between elastic stretching and force-enhanced detachment

3. **Force Dependence**: Higher nu → stronger overshoot, earlier peak time

4. **Bridge Depletion**: f_B decreases during startup, reducing effective modulus

5. **Timescale Control**: tau_b sets overshoot time, tau_a/tau_b controls re-equilibration

6. **Shear Rate Sensitivity**: Higher gamma_dot → earlier overshoot, higher peak stress (until saturation)

7. **Physical Consistency**: Verify overshoot ratio > 1, peak time ~ tau_b, f_B_ss < f_B_eq